## Importing of Libraries

In [1]:
import time
from copy import deepcopy

import numpy as np
import torch
from dlordinal.datasets import FGNet
from dlordinal.losses import ExponentialRegularisedCrossEntropyLoss
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight
from torch import cuda, nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor
from dlordinal.estimator import PytorchEstimator

## Load and preprocess of FGNet dataset

First, we present the configuration parameters for the experimentation and the number of workers for the *DataLoader*, which defines the number of subprocesses to use for data loading. In this specific case, it refers to the images.

In [2]:
optimiser_params = {
    'lr': 1e-3,
    'bs': 200,
    'epochs': 5,
    's': 2,
    'c': 0.2,
    'beta': 0.5
}

workers = 3

Now use the *FGNet* method to download and preprocess the images. Once that is done with the training data, we create a validation partition comprising 15% of the data using the *StratifiedShuffleSplit* method. Finally, with all the partitions, we load the images using a method called *DataLoader*.

In [3]:
fgnet = FGNet(root="./datasets/fgnet", download=True, process_data=True)

train_data = ImageFolder(
    root="./datasets/fgnet/FGNET/train", transform=Compose([ToTensor()])
)
test_data = ImageFolder(
    root="./datasets/fgnet/FGNET/test", transform=Compose([ToTensor()])
)

num_classes = len(train_data.classes)
classes = train_data.classes
targets = train_data.targets

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

# Create dataloaders
train_dataloader = DataLoader(
    train_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
test_dataloader = DataLoader(
    test_data, batch_size=optimiser_params["bs"], shuffle=False, num_workers=workers
)

# Get image shape
img_shape = None
for X, _ in train_dataloader:
    img_shape = list(X.shape[1:])
    break
print(f"Detected image shape: {img_shape}")

# Define class weights for imbalanced datasets
class_weights = class_weight.compute_class_weight(
    "balanced", classes=[int(c) for c in classes], y=targets
)
print(f"{class_weights=}")
class_weights = (
    torch.from_numpy(class_weights).float().to(device)
)  # Transform to Tensor

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cuda device
Detected image shape: [3, 128, 128]
class_weights=array([1.60843373, 0.55394191, 1.02692308, 0.78070175, 1.12184874,
       2.34210526])


## Model

We are using a pretrained *ResNet* model, which has previously been trained on ImageNet. We are modifying the last fully connected layer to match the number of classes in our dataset.

Finally, we define the *Adam* optimiser, which is used to adjust the network's weights and minimize the error of a loss function.

In [4]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Optimizer and scheduler
optimizer = Adam(model.parameters(), lr=optimiser_params['lr'])

## Loss Function

This exponential loss function is based on the introduction of the $L_p$, which means that there is an extra tunable parameter that can be adjusted by the learning algorithm. 

$$
f_j(p, q) = e^{-|j - q|^p}, \ 1 \leq p \leq 2
$$

where $j = \mathcal{O} (\mathcal{C}_j)$, $q = \mathcal{O} (\mathcal{C}_q)$ and p parameter can be tweaked manually or cross-validated. Hence, the p parameter controls how much a pattern is penalised when it is classified in class $\mathcal{C}_j$ and its real class is $\mathcal{C}_q$.

In [5]:
loss_fn = ExponentialRegularisedCrossEntropyLoss(num_classes=num_classes).to(device)

## Estimator

In [ ]:
estimator = PytorchEstimator(
    model=model, loss_fn=loss_fn, optimizer=optimizer, device=device, max_iter=5
)

In [ ]:
estimate = estimator.fit(train_dataloader)

Training ...
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [ ]:
print(estimator.predict_proba(test_dataloader))

Predicting ...
tensor([[ 5.4411,  3.4726,  2.1251, -1.0145, -4.4096, -4.9868],
        [ 4.4464,  2.7000,  2.1472, -0.9296, -3.9113, -5.2487],
        [ 3.9304,  3.6262,  2.4497, -1.3358, -4.2533, -5.5883],
        ...,
        [-2.6135, -3.0116, -0.4865, -1.0609,  2.2993,  2.4897],
        [ 0.5993,  1.9221,  0.4639, -1.1800, -0.9429, -1.8807],
        [-2.3832, -1.2392, -0.6678, -0.5411,  1.5725,  0.9038]],
       device='cuda:0', grad_fn=<CatBackward0>)


In [ ]:
print(estimator.predict(test_dataloader))

Predicting ...
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 2, 0,
        1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 2,
        2, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 2, 2, 3, 3, 1, 2, 2, 1, 0, 1, 4, 1,
        1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 2, 4, 4, 4, 1, 1, 1, 1, 1, 1, 4, 1, 4,
        3, 1, 4, 4, 4, 3, 1, 4, 2, 2, 4, 4, 1, 2, 4, 4, 1, 1, 1, 2, 4, 1, 1, 1,
        4, 2, 1, 4, 1, 1, 4, 1, 2, 1, 1, 3, 2, 2, 4, 4, 4, 4, 2, 4, 1, 4, 5, 2,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 1, 4, 5, 4,
        4, 5, 5, 4, 4, 0, 5, 1, 4], device='cuda:0')
